In [70]:
# Ячейка 1: Импорты, настройки и пути
import pandas as pd
import json
import random
import os
from tqdm.auto import tqdm
import pickle
import time
import csv
from typing import List, Dict, Any


# Пути
goods_path = "data/goods/goods_data_100k.parquet"
selected_posts_path = "data/posts/ajtkulov/selected/selected500k_cleaned.jsonl"
categories_mapping_path = "data/catgory_mapping_actual.json"

# Папка для результатов и чекпоинтов
output_dir = "data/silver"
checkpoint_dir = "data/silver/checkpoints"
os.makedirs(output_dir, exist_ok=True)
os.makedirs(checkpoint_dir, exist_ok=True)

# Параметры
pairs_per_csv = 1000000          # сколько пар в одном csv
checkpoint_every = 5000            # сохранять прогресс каждые N постов
max_total_pairs = 5000000        # лимит пар всего

# Файлы чекпоинта
checkpoint_file = os.path.join(checkpoint_dir, "resume_state.pkl")
processed_posts_checkpoint = os.path.join(checkpoint_dir, "processed_posts_ids.pkl")



In [71]:
# Заглушка для cross-encoder
#  !!! здесь функция для обученного на goldern cross encoder
# поставит каждой паре реальный score 
def get_cross_encoder_score(post_text, product_desc):
    return round(random.uniform(0, 1), 4)

Загружает данные о товарах из файла Parquet, очищает поле description (убирает пропуски и заменяет переносы строк на пробелы),
а затем группирует описания товаров по корневым категориям (subj_root_name) в удобный словарь для дальнейшего быстрого доступа.

In [72]:
# Ячейка 2: Загрузка товаров и группировка
print("Загрузка товаров...")
goods_df = pd.read_parquet(goods_path)

clean_df = goods_df.copy()

# Группируем для быстрого доступа по категориям
clean_df['description'] = (
    clean_df['description']
    .fillna('')
    .str.replace('\n', ' ', regex=False)
)

goods_by_category = (
    clean_df
    .groupby('subj_root_name')['description']
    .apply(list)
    .to_dict()
)
print(f"Загружено {len(goods_df):,} товаров")
print(f"Категорий товаров: {len(goods_by_category):,}")
print(goods_by_category['Автомобильные товары'][:3])

Загрузка товаров...
Загружено 100,000 товаров
Категорий товаров: 64
['Бескаркасные щётки стеклоочистителя SKYWAY обеспечивают равномерное очищение всей поверхности стекла. Благодаря технологическому усовершенствованию щетки SKYWAY способны удалять даже самые мелкие частицы воды. Натуральный каучук, из которого произведены щетки, и графитовое напыление позволяют значительно снизить уровень шума, а также увеличивают срок службы. И это еще не все: щетка скользит по поверхности с минимальным усилием, а резина остается эластичной в жару, мороз и даже под действием химических реагентов. Щетка плотно прилегает к стеклу даже на большой скорости, надежно защищая его от загрязнений. Благодаря простоте установки адаптера, 90 % чистящей поверхности дворника остаётся рабочей даже на высокой скорости. Точечное давление на дворник в нескольких местах обеспечивает на 30% более плотный контакт дворника со стеклом.', 'Клемма крокодил 2шт. 30А улучшают токопроводимость в Вашем автомобиле. Крокодилы - это

In [73]:
# Ячейка 3: Загрузка постов с использованием вашей функции

def load_jsonl_to_dicts(input_file: str) -> List[Dict[str, Any]]:
    """Загружает отобранные посты из jsonl файла"""
    data = []
    try:
        with open(input_file, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                line = line.strip()
                if line:  # пропускаем пустые строки
                    try:
                        item = json.loads(line)
                        # Сразу фильтруем только посты с текстом и категорией
                        if 'text' in item and 'category' in item and item['text'].strip():
                            data.append({
                                'post_text': item['text'],
                                'post_category': item['category']
                            })
                    except json.JSONDecodeError as e:
                        print(f"Ошибка парсинга JSON в строке {line_num}: {e}")
                        print(f"Проблемная строка: {line[:100]}...")
                        continue
        print(f"✓ Загружено {len(data):,} постов из {input_file}")
        return data
    except FileNotFoundError:
        print(f"✗ Файл не найден: {input_file}")
        return []
    except Exception as e:
        print(f"✗ Ошибка при чтении файла {input_file}: {e}")
        return []

# Загружаем посты
posts = load_jsonl_to_dicts(selected_posts_path)
print(posts[:3])

✓ Загружено 499,855 постов из data/posts/ajtkulov/selected/selected500k_cleaned.jsonl
[{'post_text': 'Кто еще не майнит Bull Run, переходим в одно из самых перспективных майнинг приложений - t.me/bullapp_botИксы обеспечены Завтра подробно расскажу, что я о них думаю и какого результата стоит ожидать.', 'post_category': 'Криптовалюты'}, {'post_text': 'Думал мне одному пришло, а оказывается, многим закинуло. Что смешно, мне даже сообщение никакое не написал, письмо пустое, а Бебсу целый сиз энд десист.Имейл полностью совпадает с тем что отправлял страйки iXBT и VGTimes в прошлом году. Я даже не ютубер по сталкеру, лол. Ютубер нарушает DMCA игрового разработчика тем что снял обзор на игру, так и запишем.', 'post_category': 'Юмор и развлечение'}, {'post_text': 'В мастерской Steam появились первые карты для тренировки в CS2, подборка лучших карт: Aim Botz — новая версия Aim Botz (ссылка) Training Map — новая версия CSGOHUB (ссылка) Crosshair Generator — настройка прицела (ссылка) Training H

In [74]:
# Ячейка 4: Загрузка и инверсия маппинга категорий
print("Загрузка маппинга категорий...")
with open(categories_mapping_path, 'r', encoding='utf-8') as f:
    category_mapping = json.load(f)

# Инвертируем маппинг: из категории поста -> список категорий товаров
post_to_goods_cats = {}
for goods_cat, post_cats_list in category_mapping.items():
    for post_cat in post_cats_list:
        if post_cat not in post_to_goods_cats:
            post_to_goods_cats[post_cat] = set()
        post_to_goods_cats[post_cat].add(goods_cat)

post_to_goods_cats = {k: list(v) for k, v in post_to_goods_cats.items()}

print(f"Маппинг покрывает {len(post_to_goods_cats)} категорий постов")


Загрузка маппинга категорий...
Маппинг покрывает 25 категорий постов


In [75]:
# Ячейка 5: Восстановление состояния из чекпоинтов

start_idx = 0
processed_ids = set()

if os.path.exists(checkpoint_file):
    try:
        with open(checkpoint_file, 'rb') as f:
            state = pickle.load(f)
            start_idx = state.get('last_processed_idx', 0)
        print(f"→ Возобновление с индекса {start_idx}")
    except Exception as e:
        print(f"Checkpoint повреждён ({e}), начинаем с начала")

if os.path.exists(processed_posts_checkpoint):
    try:
        with open(processed_posts_checkpoint, 'rb') as f:
            processed_ids = pickle.load(f)
        print(f"Уже обработано {len(processed_ids)} постов")
    except Exception as e:
        print(f"Не удалось загрузить processed_ids ({e})")

# Определяем следующий номер файла для сохранения
file_counter = 1
existing_files = [f for f in os.listdir(output_dir) if f.startswith("silver_pairs_part_")]
if existing_files:
    nums = []
    for f in existing_files:
        try:
            num = int(f.replace("silver_pairs_part_", "").replace(".csv", ""))
            nums.append(num)
        except:
            continue
    if nums:
        file_counter = max(nums) + 1
print(f"Следующий файл будет: silver_pairs_part_{file_counter}.csv")

Следующий файл будет: silver_pairs_part_1.csv


### Основной цикл генерации серебряных пар с fallback-механизмом

- Проходит по всем постам (с tqdm-прогресс-баром и поддержкой возобновления с чекпоинта)
- Для каждого поста пытается найти релевантные категории товаров по заранее подготовленному маппингу `post_to_goods_cats`
- Если маппинг найден → берёт случайное описание товара из подходящей категории и оценивает релевантность через cross-encoder
- Если маппинга нет → fallback: берёт 1–3 случайных товара из всего датасета (с пометкой random fallback)
- Собранные пары (пост + описание + score + категории) накапливаются в список `all_pairs`
- Каждые `checkpoint_every` постов сохраняет состояние (последний индекс + множество обработанных id)
- Когда накопилось ≥ `pairs_per_csv` пар — сохраняет батч в отдельный CSV (`silver_pairs_part_*.csv`)
- В конце сохраняет остаток, если он остался, и выводит итоговую статистику
- Использует `utf-8-sig` + `quoting=csv.QUOTE_ALL` для корректной работы с русским текстом и кавычками

In [76]:
# Ячейка 6: Основной цикл с fallback на случайные товары

all_pairs = []
total_pairs_generated = 0

pbar = tqdm(range(start_idx, len(posts)), initial=start_idx, total=len(posts), desc="Посты")

for idx in pbar:
    if idx in processed_ids:
        continue

    post = posts[idx]
    post_cat = post['post_category']
    relevant_goods_cats = post_to_goods_cats.get(post_cat, None)

    selected_for_this_post = []

    if relevant_goods_cats:  # есть маппинг
        for goods_cat in relevant_goods_cats:
            if goods_cat in goods_by_category and goods_by_category[goods_cat]:
                desc = random.choice(goods_by_category[goods_cat])
                score = get_cross_encoder_score(post['post_text'], desc)
                selected_for_this_post.append({
                    'product_desc': desc,
                    'post_text': post['post_text'],
                    'score': score,
                    'post_category': post_cat,
                    'goods_category': goods_cat
                })
    else:
        # Fallback: нет маппинга → 1–3 случайных товара из всего датасета
        num_random = random.randint(1, 3)
        for _ in range(num_random):
            if not goods_df.empty:
                row = goods_df.sample(1).iloc[0]
                desc = row['description']
                cat_name = row['subj_root_name']
                score = get_cross_encoder_score(post['post_text'], desc)
                selected_for_this_post.append({
                    'product_desc': desc,
                    'post_text': post['post_text'],
                    'score': score,
                    'post_category': post_cat,
                    'goods_category': f"{cat_name} (random fallback)"
                })

    if selected_for_this_post:
        all_pairs.extend(selected_for_this_post)
        total_pairs_generated += len(selected_for_this_post)

    processed_ids.add(idx)

    # Чекпоинт
    if (idx + 1) % checkpoint_every == 0 or idx == len(posts) - 1:
        state = {
            'last_processed_idx': idx + 1, 
            'total_pairs_generated': total_pairs_generated
        }
        with open(checkpoint_file, 'wb') as f:
            pickle.dump(state, f)
        with open(processed_posts_checkpoint, 'wb') as f:
            pickle.dump(processed_ids, f)

        pbar.set_postfix(pairs=total_pairs_generated, idx=idx)

    # Сохранение батчей
    while len(all_pairs) >= pairs_per_csv and total_pairs_generated <= max_total_pairs:
        batch = all_pairs[:pairs_per_csv]
        df = pd.DataFrame(batch)
        path = os.path.join(output_dir, f"silver_pairs_part_{file_counter}.csv")
        # ИСПРАВЛЕНИЕ: сохраняем без лишних escape-символов
        df.to_csv(path, index=False, encoding='utf-8-sig', 
                quoting=csv.QUOTE_ALL, sep='\t'
                  )
        print(f"💾 Сохранено: {path}  ({len(batch):,} пар | всего: {total_pairs_generated:,})")
        all_pairs = all_pairs[pairs_per_csv:]
        file_counter += 1

        if total_pairs_generated >= max_total_pairs:
            print(f"🎯 Достигнут лимит {max_total_pairs:,} пар.")
            break

# Сохраняем остаток
if all_pairs and total_pairs_generated <= max_total_pairs:
    df = pd.DataFrame(all_pairs)
    path = os.path.join(output_dir, f"silver_pairs_part_{file_counter}.csv")
    df.to_csv(path, index=False, encoding='utf-8-sig', 
            quoting=csv.QUOTE_ALL, sep='\t'
              )
    print(f"💾 Остаток сохранён: {path}  ({len(df):,} пар)")

print("\n✅ Готово.")
print(f"📊 Всего пар: {total_pairs_generated:,}")
print(f"📁 Папка: {output_dir}")

Посты:   0%|          | 0/499855 [00:00<?, ?it/s]

KeyboardInterrupt: 

- Создаёт случайную выборку 5000 строк (или весь набор, если меньше) и сохраняет отдельно как `_sample_5k.csv` для визуального контроля

In [77]:
# Ячейка 7: Сохранить текущее состояние + выборку 5k для проверки

# Настройки pandas для красивого вывода
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', 100)
# pd.set_option('display.width', 1000)
# pd.set_option('display.expand_frame_repr', False)

pd.set_option('display.max_colwidth', 100)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.width', None)
if 'all_pairs' not in globals() or not all_pairs:
    print("Список all_pairs пуст или не существует. Нечего сохранять.")
else:
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    base_name = f"silver_pairs_current_{timestamp}"
    
    # Создаём датафрейм
    df_current = pd.DataFrame(all_pairs)
    
    print(f"\n📊 Всего пар в памяти сейчас: {len(df_current):,}")
    print(f"📋 Столбцы: {list(df_current.columns)}")
    
    # ──────────────────────────────────────────────────────────────
    # 1. Полный набор — CSV с табуляцией, все поля в кавычках
    # ──────────────────────────────────────────────────────────────
    full_path = os.path.join(output_dir, f"{base_name}.csv")
    
    df_current.to_csv(
        full_path,
        sep='\t',
        index=False,
        encoding='utf-8-sig',
        quoting=csv.QUOTE_ALL
    )
    
    print(f"\n💾 Полный набор сохранён: {full_path}")
    print(f"   Строк: {len(df_current):,}")
    print(f"   Разделитель: табуляция (\\t) | все поля в кавычках")
    
    # ──────────────────────────────────────────────────────────────
    # 2. Случайная выборка ~5000 строк для быстрого просмотра
    # ──────────────────────────────────────────────────────────────
    sample_path = os.path.join(output_dir, f"{base_name}_sample_5k.csv")
    
    if len(df_current) >= 5000:
        df_sample = df_current.sample(n=5000, random_state=42).reset_index(drop=True)
        sample_size = 5000
    else:
        df_sample = df_current.copy()
        sample_size = len(df_sample)
        print(f"⚠️ Пар меньше 5000 → сохранён весь набор как sample ({sample_size} строк)")
    
    df_sample.to_csv(
        sample_path,
        sep='\t',
        index=False,
        encoding='utf-8-sig',
        quoting=csv.QUOTE_ALL
    )
    
    print(f"💾 Выборка сохранена: {sample_path}")
    print(f"   Строк в выборке: {sample_size:,}")
    
    # ──────────────────────────────────────────────────────────────
    # 3. ПОКАЗЫВАЕМ КРАСИВЫЕ ТАБЛИЦЫ PANDAS
    # ──────────────────────────────────────────────────────────────
    
    print("\n" + "="*80)
    print("🐼 ПЕРВЫЕ 30 СТРОК ДАТАФРЕЙМА (Pandas таблица):")
    print("="*80)
    display(df_sample.head(30))  # display покажет красивую HTML таблицу в Jupyter
    
    print("\n" + "="*80)
    print("📊 ИНФОРМАЦИЯ О ДАТАФРЕЙМЕ:")
    print("="*80)
    print(f"Всего строк: {len(df_sample)}")
    print(f"Всего колонок: {len(df_sample.columns)}")
    print(f"Типы данных:")
    print(df_sample.dtypes)
    
    print("\n" + "="*80)
    print("📈 СТАТИСТИКА ПО SCORE:")
    print("="*80)
    print(df_sample['score'].describe())
    
    print("\n" + "="*80)
    print("🏷️ РАСПРЕДЕЛЕНИЕ ПО КАТЕГОРИЯМ ПОСТОВ (топ-10):")
    print("="*80)
    print(df_sample['post_category'].value_counts().head(10))
    
    print("\n" + "="*80)
    print("🏷️ РАСПРЕДЕЛЕНИЕ ПО КАТЕГОРИЯМ ТОВАРОВ (топ-10):")
    print("="*80)
    print(df_sample['goods_category'].value_counts().head(10))
    
    print("\n✅ Готово. Файлы сохранены, таблицы показаны.")


📊 Всего пар в памяти сейчас: 398,702
📋 Столбцы: ['product_desc', 'post_text', 'score', 'post_category', 'goods_category']

💾 Полный набор сохранён: data/silver\silver_pairs_current_20260228_160647.csv
   Строк: 398,702
   Разделитель: табуляция (\t) | все поля в кавычках
💾 Выборка сохранена: data/silver\silver_pairs_current_20260228_160647_sample_5k.csv
   Строк в выборке: 5,000

🐼 ПЕРВЫЕ 30 СТРОК ДАТАФРЕЙМА (Pandas таблица):


,product_desc,post_text,score,post_category,goods_category
0,"Яркое, красивое колье из натуральных камней; варисцита, вулканической лавы и гематита. Состав: в...",Еще вот подскажите есть ли в тби мастер классы по ювелирке но не такие чтобы просто полосочку сд...,0.5303,Блоги,Бижутерия
1,Union PUBG Mobile D9 уникальное решение для настоящих любителей игр. Изготовлен с качественного ...,"Присоединяйтесь к просмотру Игр Будущего! Фиджитал спорт — это уникальная дисциплина, в которой ...",0.3347,Игры,Игровые консоли и игры
2,Держатель для соски и прорезыватель поможет малышу справится с нелегким периодом прорезывания. Р...,"Мы любим их больше всего на свете и готовы делать все возможное, чтобы они были счастливы. Мы ра...",0.1316,Семья и дети,Товары для малышей
3,**Общие данные:**Габариты: -х59х51 смРабочая поверхность: нержавеющая стальКоличество конфорок: ...,Белым по черному — ищем людей с чувством юмора.Этот канал настоящая находка для людей с острым ч...,0.0676,Технологии,Крупная бытовая техника
4,Объем (л): 25 Мощность микроволн (вт): 900 Страна-производитель: Китай Гарантия: 2 года Тип упра...,"Всем, кто не хочет угробить остатки нервной системы (а они понадобятся в ближайшие годы, судя по...",0.6058,Блоги,Техника для кухни
5,"Яблочный 100% сок прямого отжима, без сахара, без красителей, без консервантов, без ГМО. Яблоки ...","Вы не начинаете вести соцсети, потому что не понимаете че конкретно там делатьХотя по факту соцс...",0.7384,Блоги,Детское питание
6,"Зарядное устройство для ноутбука Sony Vaio. Выходное напряжение - 16 Вольт, максимальный ток наг...","Завтра все по-плану. Для затравочки скажу, что ролик имеет отношение к вышедшему ранее на канале...",0.2086,Блоги,Ноутбуки и компьютеры
7,Ботильоны на шнуровке в викторианском стиле. В качестве подкладки использовался мягкий текстиль....,А вот это реально уже Максимальную сумму IT-ипотеки собираются снизить с 18 млн до 9 млн рублей ...,0.7306,Блоги,Обувь
8,Матрас Nastella Eco Baby 115х60 см. – универсальное решение для вашей детской с первых дней жизн...,"Когда на рынке ДАМП, в зале нужно делать ПАМП Надеюсь вы откупили альту, на 20%-25% от свободных...",0.2498,Криптовалюты,Товары для малышей (random fallback)
9,Гермомешок серии СПОРТ и ТУРИЗМСовременный дизайн для любителей активного спорта и отдыха на вод...,Виктор Гусев в момент теракта находился в здании концертного зала «Крокус Сити Холл»По информаци...,0.3449,Спорт,Спортивные аксессуары



📊 ИНФОРМАЦИЯ О ДАТАФРЕЙМЕ:
Всего строк: 5000
Всего колонок: 5
Типы данных:
product_desc       object
post_text          object
score             float64
post_category      object
goods_category     object
dtype: object

📈 СТАТИСТИКА ПО SCORE:
count    5000.000000
mean        0.502584
std         0.291043
min         0.000300
25%         0.252775
50%         0.502750
75%         0.757050
max         0.999900
Name: score, dtype: float64

🏷️ РАСПРЕДЕЛЕНИЕ ПО КАТЕГОРИЯМ ПОСТОВ (топ-10):
post_category
Блоги                3825
Мода и красота        336
Здоровье и фитнес     117
Технологии            112
Еда и кулинария        62
Игры                   58
Софт и приложения      45
Дизайн                 42
Криптовалюты           38
Спорт                  34
Name: count, dtype: int64

🏷️ РАСПРЕДЕЛЕНИЕ ПО КАТЕГОРИЯМ ТОВАРОВ (топ-10):
goods_category
Игровые консоли и игры    111
Спортивная обувь          104
Спецодежда и СИЗы         102
Бижутерия                 100
Игрушки                   